In [ ]:
"""
INPUT:
- Đọc PPTX → DataFrame (x, y, width, height) theo mm
- Tạo grid raster
- Lấy doors cho Entrance/Cashier
- Trả ra mapping x,y theo Category (không lọc trùng; bạn đảm bảo mỗi Category chỉ 1 shape)
"""

import math
import numpy as np
import pandas as pd
from pptx import Presentation
from pathlib import Path
from typing import Dict, Tuple, Set
from src.config import RAW_DATA_DIR


def _EMU_PER_MM():
    return 914400 / 25.4


def load_shapes_from_ppt_mm(ppt_path: Path) -> pd.DataFrame:
    if not ppt_path.exists():
        raise FileNotFoundError(f"Không tìm thấy file layout: {ppt_path}")
    prs = Presentation(ppt_path)
    emu = _EMU_PER_MM()
    rows = []
    for slide in prs.slides:
        for sh in slide.shapes:
            if sh.has_text_frame and sh.text_frame.text.strip():
                txt = sh.text_frame.text.strip()
                rows.append(
                    {
                        "Category": txt,
                        "x": sh.left / emu,
                        "y": sh.top / emu,
                        "width": sh.width / emu,
                        "height": sh.height / emu,
                    }
                )
    if not rows:
        raise RuntimeError("Không đọc được shape nào từ PPT.")
    return pd.DataFrame(rows)


def estimate_cell_size_from_layout(df: pd.DataFrame) -> float:
    vals = []
    arr = df[["x", "y", "width", "height"]].values
    n = len(arr)
    for i in range(n):
        x0, y0, w0, h0 = arr[i]
        x0b, y0b = x0 + w0, y0 + h0
        for j in range(i + 1, n):
            x1, y1, w1, h1 = arr[j]
            x1b, y1b = x1 + w1, y1 + h1
            if not (y0b < y1 or y1b < y0):
                gx = max(0, x0 - x1b, x1 - x0b)
                if gx > 1:
                    vals.append(gx)
            if not (x0b < x1 or x1b < x0):
                gy = max(0, y0 - y1b, y1 - y0b)
                if gy > 1:
                    vals.append(gy)
    if vals:
        return max(1, int(min(vals)) // 2)
    min_dim = np.minimum(df["width"], df["height"])
    return (
        max(1, int(np.median(min_dim[min_dim > 0]) / 4)) if (min_dim > 0).any() else 5
    )


def rasterize_grid(df: pd.DataFrame, cell_size: float, padding_ratio: float = 0.06):
    cats = list(df["Category"].astype(str).unique())
    name2id = {c: i + 1 for i, c in enumerate(cats)}
    id2name = {v: k for k, v in name2id.items()}

    x0, y0 = df["x"].min(), df["y"].min()
    x1, y1 = (df["x"] + df["width"]).max(), (df["y"] + df["height"]).max()
    pad_x, pad_y = int((x1 - x0) * padding_ratio), int((y1 - y0) * padding_ratio)
    min_x, min_y = x0 - pad_x, y0 - pad_y
    max_x, max_y = x1 + pad_x, y1 + pad_y

    W = int(math.ceil((max_x - min_x) / cell_size))
    H = int(math.ceil((max_y - min_y) / cell_size))
    if W * H > 1e7:
        scale = ((W * H) / 1e7) ** 0.5
        cell_size *= scale
        W = int(math.ceil((max_x - min_x) / cell_size))
        H = int(math.ceil((max_y - min_y) / cell_size))

    grid = np.zeros((H, W), dtype=np.int32)
    for _, r in df.iterrows():
        did = name2id[str(r["Category"])]
        gx0 = int(math.floor((r["x"] - min_x) / cell_size))
        gx1 = int(math.ceil((r["x"] + r["width"] - min_x) / cell_size))
        gy0 = int(math.floor((r["y"] - min_y) / cell_size))
        gy1 = int(math.ceil((r["y"] + r["height"] - min_y) / cell_size))
        grid[gy0:gy1, gx0:gx1] = did

    meta = {"min_x": min_x, "min_y": min_y, "cell_size": cell_size, "W": W, "H": H}
    return grid, name2id, id2name, meta


def door_nodes_for_dept(grid: np.ndarray, dept_id: int) -> Set[Tuple[int, int]]:
    H, W = grid.shape
    doors = set()
    ys, xs = np.where(grid == dept_id)
    for y, x in zip(ys, xs):
        for ny, nx in ((y - 1, x), (y + 1, x), (y, x - 1), (y, x + 1)):
            if 0 <= ny < H and 0 <= nx < W and grid[ny, nx] == 0:
                doors.add((ny, nx))
    return doors


def build_inputs(layout_pptx: str = "layout.pptx"):
    df = load_shapes_from_ppt_mm(RAW_DATA_DIR / layout_pptx)

    # Kiểm tra tồn tại đúng tên
    if "Entrance" not in set(df["Category"]):
        raise RuntimeError("Thiếu 'Entrance' trong PPT.")
    if "Cashier" not in set(df["Category"]):
        raise RuntimeError("Thiếu 'Cashier' trong PPT.")

    # Grid
    cell_size = estimate_cell_size_from_layout(df)
    grid, name2id, id2name, meta = rasterize_grid(df, cell_size)

    # Doors
    ent_doors = door_nodes_for_dept(grid, name2id["Entrance"])
    cas_doors = door_nodes_for_dept(grid, name2id["Cashier"])

    # Mapping x,y theo Category — KHÔNG lọc trùng (bạn đảm bảo duy nhất)
    xy_map = df.set_index("Category")[["x", "y"]].to_dict(orient="index")

    return {
        "df": df,
        "grid": grid,
        "name2id": name2id,
        "id2name": id2name,
        "meta": meta,
        "ent_doors": ent_doors,
        "cas_doors": cas_doors,
        "xy_map": xy_map,
        "cell_size": cell_size,
    }


bundle = build_inputs(layout_pptx="layout.pptx")

In [ ]:
"""
CORE: Xây dựng đường đi khách hàng
- Bắt đầu tại Entrance
- Kết thúc tại Cashier
- Không quay lại Entrance/Cashier giữa chừng
- Mặc định: phủ toàn bộ Category còn lại (ngoại trừ Entrance/Cashier)
"""

import math
from typing import Dict, Tuple, Set, List
import numpy as np
import pandas as pd
import networkx as nx
from collections import deque
from networkx.algorithms import approximation as nx_apx


def build_aisle_graph(grid: np.ndarray) -> nx.Graph:
    G = nx.Graph()
    H, W = grid.shape
    for y in range(H):
        for x in range(W):
            if grid[y, x] == 0:
                for dy, dx in ((0, 1), (1, 0)):  # tránh thêm 2 lần
                    ny, nx2 = y + dy, x + dx
                    if 0 <= ny < H and 0 <= nx2 < W and grid[ny, nx2] == 0:
                        G.add_edge((y, x), (ny, nx2), weight=1)
    return G


def nearest_open_to_dept(grid: np.ndarray, dept_id: int, max_radius: int = 15):
    H, W = grid.shape
    q = deque()
    seen = set()
    ys, xs = np.where(grid == dept_id)
    if len(ys) == 0:
        return None
    for y, x in zip(ys, xs):
        q.append(((y, x), 0))
        seen.add((y, x))
    while q:
        (y, x), d = q.popleft()
        if d >= max_radius:
            continue
        for dy, dx in ((0, 1), (0, -1), (1, 0), (-1, 0)):
            ny, nx = y + dy, x + dx
            if 0 <= ny < H and 0 <= nx < W and (ny, nx) not in seen:
                if grid[ny, nx] == 0:
                    return (ny, nx)
                seen.add((ny, nx))
                q.append(((ny, nx), d + 1))
    return None


def pick_rep_door_for_dept(grid: np.ndarray, dept_id: int, G_allowed: nx.Graph):
    # cửa kề trực tiếp
    H, W = grid.shape
    doors = set()
    ys, xs = np.where(grid == dept_id)
    for y, x in zip(ys, xs):
        for ny, nx in ((y - 1, x), (y + 1, x), (y, x - 1), (y, x + 1)):
            if 0 <= ny < H and 0 <= nx < W and grid[ny, nx] == 0:
                if (ny, nx) in G_allowed:
                    doors.add((ny, nx))
    if doors:
        # trung vị hình học (đơn giản: chọn điểm gần trung bình)
        ys = [p[0] for p in doors]
        xs = [p[1] for p in doors]
        cy, cx = int(np.mean(ys)), int(np.mean(xs))
        return min(doors, key=lambda p: (p[0] - cy) ** 2 + (p[1] - cx) ** 2)

    # nếu không có, dùng điểm mở gần nhất
    rep = nearest_open_to_dept(grid, dept_id)
    return rep if rep in G_allowed else None


def shortest_path_safe(G: nx.Graph, u, v) -> List[Tuple[int, int]]:
    if u == v:
        return [u]
    try:
        return nx.shortest_path(G, u, v, weight="weight")
    except Exception:
        return []


def shortest_length_safe(G: nx.Graph, u, v) -> float:
    if u == v:
        return 0.0
    try:
        return nx.shortest_path_length(G, u, v, weight="weight")
    except Exception:
        return float("inf")


def restrict_graph(
    G: nx.Graph, blocked: Set[Tuple[int, int]], preserve: Set[Tuple[int, int]]
):
    if not blocked:
        return G
    remove = set(blocked) - set(preserve)
    if not remove:
        return G
    H = G.copy()
    H.remove_nodes_from([n for n in remove if n in H])
    return H


def compute_customer_path(
    grid: np.ndarray,
    name2id: Dict[str, int],
    id2name: Dict[int, str],
    df: pd.DataFrame,
    ent_doors: Set[Tuple[int, int]],
    cas_doors: Set[Tuple[int, int]],
    require_full_coverage: bool = True,
    max_refine_iters: int = 3,
) -> List[Tuple[int, int]]:
    """Trả ra list node (y,x) theo grid."""
    G_allowed = build_aisle_graph(grid)
    allowed_nodes = set(G_allowed.nodes())
    if not allowed_nodes:
        raise RuntimeError("Không có lối đi (pixel=0) trong grid.")

    # Danh mục cần ghé = tất cả trừ Entrance/Cashier
    regular_names = [c for c in id2name.values() if c not in ("Entrance", "Cashier")]

    # Door đại diện cho từng category
    rep_nodes = []
    target_ids = set()
    for name in regular_names:
        did = name2id[name]
        rep = pick_rep_door_for_dept(grid, did, G_allowed)
        if rep:
            rep_nodes.append(rep)
            target_ids.add(did)

    # Door đại diện cho Entrance/Cashier
    ent_rep = next(iter(ent_doors)) if ent_doors else None
    cas_rep = next(iter(cas_doors)) if cas_doors else None
    if not ent_rep or not cas_rep:
        raise RuntimeError(
            "Không xác định được node đại diện cho Entrance hoặc Cashier."
        )

    all_entry_nodes = set(ent_doors)
    all_cashier_nodes = set(cas_doors)
    G_mid = restrict_graph(
        G_allowed, blocked=all_entry_nodes | all_cashier_nodes, preserve=set()
    )

    def order_stops(stops: List[Tuple[int, int]]) -> List[Tuple[int, int]]:
        if len(stops) <= 1:
            return stops
        K = nx.Graph()
        for i, u in enumerate(stops):
            for v in stops[i + 1 :]:
                d = shortest_length_safe(G_mid, u, v)
                if math.isfinite(d):
                    K.add_edge(u, v, weight=d)
        if K.number_of_edges() == 0:
            return stops
        if nx.is_connected(K):
            return nx_apx.traveling_salesman_problem(K, cycle=False)
        ordered = []
        for comp in nx.connected_components(K):
            comp_list = list(comp)
            if len(comp_list) > 1:
                part = nx_apx.traveling_salesman_problem(
                    K.subgraph(comp_list), cycle=False
                )
            else:
                part = comp_list
            ordered.extend(part)
        return ordered

    def route_through(stops: List[Tuple[int, int]]) -> List[Tuple[int, int]]:
        if not stops:
            # chỉ ent -> cas, khoá giữa
            G_seg = restrict_graph(
                G_allowed,
                blocked=all_entry_nodes | all_cashier_nodes,
                preserve={ent_rep, cas_rep},
            )
            return shortest_path_safe(G_seg, ent_rep, cas_rep)

        middle = order_stops(stops)
        first, last = middle[0], middle[-1]
        G_ent_first = restrict_graph(
            G_allowed,
            blocked=all_cashier_nodes | (all_entry_nodes - {ent_rep}),
            preserve={ent_rep, first},
        )
        G_last_cas = restrict_graph(
            G_allowed,
            blocked=all_entry_nodes | (all_cashier_nodes - {cas_rep}),
            preserve={last, cas_rep},
        )
        dist_fwd = shortest_length_safe(
            G_ent_first, ent_rep, first
        ) + shortest_length_safe(G_last_cas, last, cas_rep)

        G_ent_last = restrict_graph(
            G_allowed,
            blocked=all_cashier_nodes | (all_entry_nodes - {ent_rep}),
            preserve={ent_rep, last},
        )
        G_first_cas = restrict_graph(
            G_allowed,
            blocked=all_entry_nodes | (all_cashier_nodes - {cas_rep}),
            preserve={first, cas_rep},
        )
        dist_rev = shortest_length_safe(
            G_ent_last, ent_rep, last
        ) + shortest_length_safe(G_first_cas, first, cas_rep)

        if dist_rev < dist_fwd:
            middle.reverse()

        chain = [ent_rep] + middle + [cas_rep]
        path = []
        for u, v in zip(chain[:-1], chain[1:]):
            G_seg = restrict_graph(
                G_allowed, blocked=all_entry_nodes | all_cashier_nodes, preserve={u, v}
            )
            seg = shortest_path_safe(G_seg, u, v)
            if not seg:
                continue
            path.extend(seg if not path else seg[1:])
        return path

    path_nodes = route_through(rep_nodes)

    if require_full_coverage:

        def seen_ids(coords: List[Tuple[int, int]], neighbor_range=4):
            H, W = grid.shape
            s = set()
            path_set = set(coords)
            for y, x in path_set:
                for dy in range(-neighbor_range, neighbor_range + 1):
                    for dx in range(-neighbor_range, neighbor_range + 1):
                        ny, nx = y + dy, x + dx
                        if 0 <= ny < H and 0 <= nx < W and grid[ny, nx] > 0:
                            s.add(grid[ny, nx])
            return s

        for _ in range(max_refine_iters):
            missed = (
                target_ids
                - seen_ids(path_nodes, neighbor_range=5)
                - {name2id["Entrance"], name2id["Cashier"]}
            )
            if not missed:
                break
            # thêm các điểm đại diện còn thiếu
            for did in list(missed):
                rep = pick_rep_door_for_dept(grid, did, G_allowed)
                if rep and rep not in rep_nodes:
                    rep_nodes.append(rep)
            path_nodes = route_through(rep_nodes)

    return path_nodes


def sequence_from_path(
    grid: np.ndarray,
    path_coords: List[Tuple[int, int]],
    id2name: Dict[int, str],
    neighbor_range: int = 3,
) -> pd.DataFrame:
    """
    Trả về DataFrame: Step, Dept_ID, Category
    (KHÔNG tính x,y ở đây)
    """
    H, W = grid.shape
    neigh = [
        (dy, dx)
        for dy in range(-neighbor_range, neighbor_range + 1)
        for dx in range(-neighbor_range, neighbor_range + 1)
    ]

    raw = []
    for y, x in path_coords:
        adj = []
        for dy, dx in neigh:
            ny, nx = y + dy, x + dx
            if 0 <= ny < H and 0 <= nx < W and grid[ny, nx] > 0:
                adj.append(grid[ny, nx])
        if adj:
            vals, cnts = np.unique(adj, return_counts=True)
            raw.append(int(vals[np.argmax(cnts)]))

    if not raw:
        return pd.DataFrame(columns=["Step", "Dept_ID", "Category"])

    # nén chuỗi (bỏ trùng liên tiếp)
    comp = [raw[0]]
    for v in raw[1:]:
        if v != comp[-1]:
            comp.append(v)

    # loại Entrance/Cashier ở giữa (giữ đầu/cuối nếu trùng)
    final = []
    for i, did in enumerate(comp):
        name = id2name.get(did, "")
        if i == 0 or i == len(comp) - 1:
            final.append(did)
            continue
        if name not in ("Entrance", "Cashier"):
            final.append(did)

    # re-compress
    seq = [final[0]] if final else []
    for v in final[1:]:
        if v != seq[-1]:
            seq.append(v)

    out = [
        {"Step": i, "Dept_ID": did, "Category": id2name.get(did, f"ID {did}")}
        for i, did in enumerate(seq, 1)
    ]
    return pd.DataFrame(out)

In [ ]:
"""
OUTPUT:
- Chạy core để lấy path
- Xuất CSV: Step, Dept_ID, Category, x, y (x,y merge trực tiếp từ df gốc)
- Vẽ PNG trực quan (giữ nguyên)
"""

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, List

from src.config import OUTPUT_DATA_DIR
from src.step_core import compute_customer_path, sequence_from_path


def visualize(
    grid: np.ndarray,
    id2name: Dict[int, str],
    path_coords: List[Tuple[int, int]],
    output_png: Path,
) -> None:
    H, W = grid.shape
    fig, ax = plt.subplots(figsize=(min(30, 18 * (W / max(1, H))), min(30, 18)))
    unique_ids = np.unique(grid)
    max_id = int(unique_ids.max()) if len(unique_ids) > 0 else 0
    colors = ["#FFFFFF"] + [
        plt.cm.get_cmap("tab20", max(1, max_id))(i) for i in range(max(1, max_id))
    ]
    cmap = mcolors.ListedColormap(colors)
    bounds = list(range(0, max_id + 2))
    norm = mcolors.BoundaryNorm(bounds, cmap.N)

    ax.imshow(grid, cmap=cmap, norm=norm, interpolation="none")

    for did in np.unique(grid[grid > 0]):
        ys, xs = np.where(grid == did)
        cx, cy = np.mean(xs), np.mean(ys)
        name = id2name.get(did, f"ID {did}")
        region_color = cmap(norm(did))
        lum = (
            0.299 * region_color[0] + 0.587 * region_color[1] + 0.114 * region_color[2]
        )
        txt_color = "white" if lum < 0.5 else "black"
        w = xs.max() - xs.min() + 1
        h = ys.max() - ys.min() + 1
        rot = 90 if h > w * 1.5 and len(name) > 5 else 0
        fontsize = max(6, min(10, int(np.sqrt(w * h) / max(1, len(name)) * 4)))
        ax.text(
            cx,
            cy,
            name,
            va="center",
            ha="center",
            color=txt_color,
            fontsize=fontsize,
            rotation=rot,
            weight="bold",
        )

    if path_coords:
        ys, xs = zip(*path_coords)
        ax.plot(xs, ys, "-", linewidth=2.0, zorder=10)
        ax.plot(xs[0], ys[0], "o", markersize=10, zorder=11, label="Start (Entrance)")
        ax.plot(xs[-1], ys[-1], "X", markersize=10, zorder=11, label="End (Cashier)")

    ax.set_title(
        "Customer Path (No re-entry Entrance/Cashier)", fontsize=16, weight="bold"
    )
    ax.grid(False)
    ax.tick_params(
        axis="both",
        which="both",
        bottom=False,
        left=False,
        labelbottom=False,
        labelleft=False,
    )
    ax.legend(loc="best")
    plt.savefig(output_png, dpi=200, bbox_inches="tight")
    plt.close(fig)


def run_and_export(
    input_bundle: Dict,
    customer_path_sequence: str = "customer_path_sequence.csv",
    customer_path_visualization: str = "customer_path_visualization.png",
):
    """
    input_bundle: dict từ layout_input.build_inputs()
    Xuất:
      - CSV: customer_path_sequence.csv (Step, Dept_ID, Category, x, y) — x,y lấy trực tiếp từ df gốc
      - PNG: customer_path_visualization.png
    """
    df = input_bundle["df"]
    grid = input_bundle["grid"]
    name2id = input_bundle["name2id"]
    id2name = input_bundle["id2name"]
    ent_doors = input_bundle["ent_doors"]
    cas_doors = input_bundle["cas_doors"]

    path_nodes = compute_customer_path(
        grid,
        name2id,
        id2name,
        df,
        ent_doors,
        cas_doors,
        require_full_coverage=True,
        max_refine_iters=3,
    )
    if not path_nodes:
        raise RuntimeError("Không thể tạo được đường đi.")

    # 1) Bảng step (chưa có x,y)
    seq_df = sequence_from_path(grid, path_nodes, id2name, neighbor_range=3)

    # 2) MERGE x,y trực tiếp từ df gốc (bạn đảm bảo không trùng Category)
    xy_all = df[["Category", "x", "y"]]
    seq_with_xy = seq_df.merge(xy_all, on="Category", how="left")

    # Xuất CSV
    seq_with_xy.to_csv(
        OUTPUT_DATA_DIR / customer_path_sequence, index=False, encoding="utf-8-sig"
    )

    # PNG
    visualize(grid, id2name, path_nodes, OUTPUT_DATA_DIR / customer_path_visualization)

    return {
        "csv": OUTPUT_DATA_DIR / customer_path_sequence,
        "png": OUTPUT_DATA_DIR / customer_path_visualization,
    }


out = run_and_export(
    input_bundle=bundle,
    customer_path_sequence="customer_path_sequence.csv",
    customer_path_visualization="customer_path_visualization.png",
)
print("Saved:", out)